<a href="https://colab.research.google.com/github/jejec0703/TIL/blob/master/%ED%8F%89%EA%B0%804_%EC%A4%80%EB%B9%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1번

In [ ]:
import numpy as np

# Write a function that takes as input a list of numbers, and returns
# the list of values given by the softmax function.
def softmax(L):
    expL = np.exp(L)
    sumExpL = sum(expL)
    result = []
    for i in expL:
      result.append((i/sumExpL).round(2))
    return result

print(softmax([2, 1, 0]))

[0.67, 0.24, 0.09]


## 2번

In [ ]:
# Implement the following functions

# Activation (sigmoid) function
def sigmoid(x):
  return 1/ (1 + np.exp(-x))

# Output (prediction) formula
def output_formula(features, weights, bias):
    return sigmoid(np.dot(features, weights) + bias)

# Error (log-loss) formula
def error_formula(y, output):
    return - y*np.log(output) - (1- y)* np.log(1 - output)

# Gradient descent step
def update_weights(x, y, weights, bias, learnrate):
  output = output_formula(x, weights, bias)
  d_error = y - output
  weights += learnrate * d_error * x
  bias += learnrate * d_error

  return weights, bias

#답 : d_error = y - output

## 3번

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

sentences = ['바나나 사과 포도 포도',
         '사과 포도',
         '포도 바나나',
         '짜장면 짬뽕 탕수육',
         '볶음밥 탕수육',
         '짜장면 짬뽕',
         '라면 스시',
         '스시',
         '가츠동 스시 소바',
         '된장찌개 김치찌개 김치',
         '김치 된장',
         '비빔밥 김치'
         ]

tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
max_sequence_len = max([len(x) for x in sentences])
sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, padding='post',  maxlen=max_sequence_len, truncating='post')

In [ ]:
# word_index = tokenizer.word_index
# print(len(word_index))
# print(word_index)

17
{'<OOV>': 1, '포도': 2, '스시': 3, '김치': 4, '바나나': 5, '사과': 6, '짜장면': 7, '짬뽕': 8, '탕수육': 9, '볶음밥': 10, '라면': 11, '가츠동': 12, '소바': 13, '된장찌개': 14, '김치찌개': 15, '된장': 16, '비빔밥': 17}


## 4번

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np 

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt
data = open('/tmp/sonnets.txt').read()

corpus = data.lower().split("\n")

--2020-08-26 12:36:14--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 172.217.214.128, 108.177.111.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘/tmp/sonnets.txt’

/tmp/sonnets.txt    100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2020-08-26 12:36:14 (109 MB/s) - ‘/tmp/sonnets.txt’ saved [93578/93578]



In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len))

In [ ]:
xs, label = input_sequences[:, :-1], input_sequences[:, -1]
label = ku.to_categorical(label, num_classes=total_words)

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
#Bidirectional(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))
#softmax
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#categorical_crossentropy
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 10, 100)           321100    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 10, 300)           301200    
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 300)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_6 (Dense)              (None, 1605)              162105    
_________________________________________________________________
dense_7 (Dense)              (None, 3211)              5156866   
Total params: 6,101,671
Trainable params: 6,101,671
Non-trainable params: 0
____________________________________________